In [1]:
import gmql as gl

In [2]:
# path of the dataset
input_path = "/home/luca/Documenti/GMQL-Python/resources/hg_narrowPeaks"
np_parser = gl.parsers.NarrowPeakParser()
dataset = gl.GMQLDataset(parser=np_parser)
dataset = dataset.load_from_path(path=input_path)

2017-03-31 22:16:36,582 - gmql_logger - INFO - loading metadata
2017-03-31 22:16:41,456 - gmql_logger - INFO - parsing metadata
2017-03-31 22:16:41,458 - gmql_logger - INFO - collecting metadata
2017-03-31 22:17:02,456 - gmql_logger - INFO - dataframe construction


100%|██████████| 115/115 [00:39<00:00,  3.03it/s]

2017-03-31 22:17:42,045 - gmql_logger - INFO - loading region data


2017-03-31 22:17:42,921 - gmql_logger - INFO - parsing region data


In [ ]:
def aggregate(aggregates):
    peaks = aggregates['peaks']
    return sum(peaks)

new_meta = {
    'strange_aggregation': aggregate
}
ext = dataset.extend(new_attr_dict=new_meta, n=10)

In [ ]:
n_part = dataset.reg_dataset.getNumPartitions()
dataset.reg_dataset.partitionBy(n_part, lambda x : x)

In [ ]:
dataset.reg_dataset = dataset.reg_dataset.cache()

In [ ]:
r = dataset.reg_dataset